In [21]:
!pip3 install diffprivlib
import diffprivlib.models as dp

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import psutil
import time
from sklearn.pipeline import Pipeline

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer

In [22]:

url = 'household_power_consumption.txt'
column_names = ['date', 'time', 'global_active_power', 'global_reactive_power', 'voltage', 'global_intensity', 'sub_metering_1', 'sub_metering_2', 'sub_metering_3']

raw_dataset = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=';', skipinitialspace=True)


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,3,4,5,6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
dataset = raw_dataset.copy()

In [24]:
dataset = dataset.dropna()
dataset = dataset.iloc[1:]
del dataset['date']
del dataset['time']
dataset.head()

,global_active_power,global_reactive_power,voltage,global_intensity,sub_metering_1,sub_metering_2,sub_metering_3
1,4.216,0.418,234.840,18.400,0.000,1.000,17.000
2,5.360,0.436,233.630,23.000,0.000,1.000,16.000
3,5.374,0.498,233.290,23.000,0.000,2.000,17.000
4,5.388,0.502,233.740,23.000,0.000,1.000,17.000
5,3.666,0.528,235.680,15.800,0.000,1.000,17.000


In [25]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)


KeyboardInterrupt: ignored

In [ ]:
X_train = train_dataset.copy()
X_test = test_dataset.copy()

X_train=X_train.astype('float')
X_test=X_test.astype('float')

y_train = X_train.pop('sub_metering_3')
y_test = X_test.pop('sub_metering_3')


In [ ]:
from sklearn.linear_model import LogisticRegression as sk_LogisticRegression

logreg = Pipeline([
    ('scaler', Normalizer()), 
    ('clf', sk_LogisticRegression()) 
])



In [ ]:
from diffprivlib.models import LinearRegression
dp_logreg = Pipeline([
    ('scaler', Normalizer()), 
    ('clf', dp.LogisticRegression(epsilon=float("inf"), data_norm=2)) 
])



In [ ]:
accuracy_final1 = []
for _ in " "*10: 
  dp_logreg.set_params(clf__epsilon=1).fit(X_train, y_train) 
  accuracy_final1.append(dp_logreg.score(X_test, y_test))


In [ ]:
accuracy_final01 = []
for _ in " "*10: 
  dp_logreg.set_params(clf__epsilon=0.1).fit(X_train, y_train) 
  accuracy_final01.append(dp_logreg.score(X_test, y_test))

In [ ]:
accuracy_final001 = []
for _ in " "*10: 
  dp_logreg.set_params(clf__epsilon=0.01).fit(X_train, y_train) 
  accuracy_final001.append(dp_logreg.score(X_test, y_test))

In [26]:
accuracy_final = []
accuracy_final1.remove(max(accuracy_final1))
accuracy_final1.remove(min(accuracy_final1))

accuracy_final01.remove(max(accuracy_final01))
accuracy_final01.remove(min(accuracy_final01))

accuracy_final001.remove(max(accuracy_final001))
accuracy_final001.remove(min(accuracy_final001))

accuracy_final.append(sum(accuracy_final1)/10)
accuracy_final.append(sum(accuracy_final01)/10)
accuracy_final.append(sum(accuracy_final001)/10)

In [27]:
accuracy_final

[0.34545070818872525, 0.32210184577223316, 0.320056748897852]